In [1]:
import requests
import random
import string
import time

In [191]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

### make creds

In [46]:
user_creds = []
batch_id = int(time.time())
for i in range(1, 21):
    name = 'user {}.{}'.format(batch_id, str(i).zfill(2))
    user_creds.append({
        'display_name': name, 
        'username': '{}@email.com'.format(name.replace(' ','.')),
        'password': 'password'
    })

### signup

In [47]:
for user_cred in user_creds:
    # get token
    r = requests.post('http://localhost:8080/auth/signup', json=user_cred)
    assert r.status_code in range(200,300), 'failed {}'.format(user_cred['username'])
    
print('success')

success


### auth

In [48]:
user_tokens = {}

for user_cred in user_creds:
    r = requests.post('http://localhost:8080/auth/login', json=user_cred)
    assert r.status_code in range(200,300), 'failed {}'.format(user_cred['username'])
    username = user_cred['username'].lower()
    user_tokens[username] = r.json()['token']
    
print('success')

success


In [199]:
random_token = random.choice(list(user_tokens.values()))
auth_header = {'authorization': 'bearer {}'.format(random_token)}
r = requests.get('http://localhost:8080/user/', headers=auth_header)
user_data = {u['user_id']: u for u in r.json()['users']}

### create posts

In [192]:
tag_names = [
    'apps',
    'aws',
    'containers',
    'docker',
    'healthtech',
    'hippa',
    'jill',
    'tutorial'
]

In [141]:
with open('/usr/share/dict/words', 'r') as f:
    words_list = f.read().strip().splitlines()

In [189]:
title = ' '.join(random.choices(words_list, k=random.randint(3,10)))
body = ''
for _ in range(random.randint(2,30)):
    sentence = ' '.join(random.choices(words_list, k=random.randint(3,10))).title() + '. '
    body += sentence
body = body.strip()

In [208]:
explicit_tags = k_unique(tag_names, k=3)

In [68]:
for i in range(10):
    post = {
        'title': "Post {}".format(i),
        'body': ''.join(random.choices(string.ascii_letters + string.digits, k=10000)) ,
        'explicit_tags': k_unique(tag_names, k=3),
        'collaborators': k_unique()
    }
    explicit_tags = set()
    while len(explicit_tags) < 3:
        explicit_tags.add(random.choice(tag_names))
    post['explicit_tags'] = list(explicit_tags)
    
    r = requests.post('http://localhost:8080/post/', json=post, headers = auth_headers)
    assert r.status_code in range(200,300), 'bad status'

In [69]:
args = {'implicit': True}

In [70]:
r = requests.get(
    'http://localhost:8080/tag/', headers=auth_headers, 
    params=args
).json()